In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:

model_name = 'meta-llama/Llama-2-7b-chat-hf'
tokenize = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [36]:
true_summary = "Theodule solicits investors for Creative Capital by guaranteeing a 100% return on their investment within 90 days based on his claimed successful trading of stocks and options."
true_text = """Prior to distributing the proceeds back to the individual club members, the investment clubs typically charge a second 10% commission on the principal.18. Since the commencement of the investment scheme, the Defendants have raised more than $23.4 million from thousands of investors nationwide.16. Theodule also routinely boasts to investors about Creative Capital’s high rates of 
return, and stresses the need to begin investing as soon as possible.Page 2 of 10 
$15.2 million collected from new investors in typical Ponzi scheme fashion.Page 4 of 10 
made millionaires out of a significant number of people in the time it had taken her to decide to invest, and pressured her to liquidate the equity in her home to invest with him.They guarantee investors 100% returns with no risk, and claim to invest in the stocks and options of well-known companies such as Google, John Deere, Monsanto, Best Buy, GameStop, and others.26. At the end of the 90-day investment period, when the Defendants have purportedly doubled the investment amount, they supposedly return the principal and profits back to the investment clubs, minus a 40% commission on the profits.21. The investment clubs pool investor funds and send them to Creative Capital for a 90-day period, during which Theodule purportedly trades stocks and options on behalf of the investment club members.15. The Defendants primarily attract investors through word-of-mouth, and Theodule makes his representations during face-to-face meetings in which he touts his ability to double investor funds in just 90 days.3. Theodule solicits investments for the Companies primarily during in-person presentations where he guarantees prospective investors a 100% return on their investment within 90 days based on his successful trading of stocks and options."""

false_summary = "Theodule solicits investors for Creative Capital by guaranteeing a 80% return on their investment within 60 days based on his claimed successful trading of stocks."

false_text = true_text

true_input_text = f"""Evaluate the compliance of a summary sentence derived from a set of sentences in a financial document. Adhere to the following verification standards:
1. Entity consistency: Check that all named entities in the summary are extracted from the source.
2. Relationship verification: Confirm that relationships between entities in the summary are present and correctly depicted in the source.
3. Directionality check: Ensure that the direction of relationships between entities in the summary matches those in the source.
4. Factual integrity: Ascertain that the summary is free from factual errors when compared to the source.
5. Entity authenticity: Confirm that the summary does not create non-existent entities.

Based on these criteria, determine if the summary sentence is a faithful representation of the source sentences. Respond with "True" if the summary complies with all standards, or "False" if it does not.

Summary sentence: {true_summary}

Source sentences: {true_text}"""

false_input_text = f"""Evaluate the compliance of a summary sentence derived from a set of sentences in a financial document. Adhere to the following verification standards:
1. Entity consistency: Check that all named entities in the summary are extracted from the source.
2. Relationship verification: Confirm that relationships between entities in the summary are present and correctly depicted in the source.
3. Directionality check: Ensure that the direction of relationships between entities in the summary matches those in the source.
4. Factual integrity: Ascertain that the summary is free from factual errors when compared to the source.
5. Entity authenticity: Confirm that the summary does not create non-existent entities.

Based on these criteria, determine if the summary sentence is a faithful representation of the source sentences. Respond with "True" if the summary complies with all standards, or "False" if it does not.

Summary sentence: {false_summary}

Source sentences: {false_text}"""


In [37]:
true_input_text = f"""
As a compliance officer at a financial institution, you're tasked with evaluating the accuracy of a summary sentence based on its alignment with source sentences from a financial document. Consider the following criteria carefully:

1. The summary accurately reflects the content of the source sentences, especially numerical information.
2. All named entities in the summary are present in the source sentences.
3. Relationships between entities in the summary are consistent with those in the source sentences.
4. The directional flow of relationships among named entities matches between the summary and source sentences.
5. There are no factual discrepancies between the summary and source sentences.
6. The summary does not introduce any entities not found in the source sentences.

Your job is to determine if the summary adheres to these criteria. Answer "Yes" if it does, or "No" if it doesn't.

Summary sentence: ```{true_summary}```

Source sentences: ```{true_text}```

Final Answer (Yes/No only): 
"""
false_input_text = f"""
As a compliance officer at a financial institution, you're tasked with evaluating the accuracy of a summary sentence based on its alignment with source sentences from a financial document. Consider the following criteria carefully:

1. The summary accurately reflects the content of the source sentences, especially numerical information.
2. All named entities in the summary are present in the source sentences.
3. Relationships between entities in the summary are consistent with those in the source sentences.
4. The directional flow of relationships among named entities matches between the summary and source sentences.
5. There are no factual discrepancies between the summary and source sentences.
6. The summary does not introduce any entities not found in the source sentences.

Your job is to determine if the summary adheres to these criteria. Answer "Yes" if it does, or "No" if it doesn't.

Summary sentence: ```{false_summary}```

Source sentences: ```{true_text}```

Final Answer (Yes/No only): 
"""

In [5]:
print(true_input_text)


As a compliance officer at a financial institution, you're tasked with evaluating the accuracy of a summary sentence based on its alignment with source sentences from a financial document. Consider the following criteria carefully:

1. The summary accurately reflects the content of the source sentences, especially numerical information.
2. All named entities in the summary are present in the source sentences.
3. Relationships between entities in the summary are consistent with those in the source sentences.
4. The directional flow of relationships among named entities matches between the summary and source sentences.
5. There are no factual discrepancies between the summary and source sentences.
6. The summary does not introduce any entities not found in the source sentences.

Your job is to determine if the summary adheres to these criteria. Answer "Yes" if it does, or "No" if it doesn't.

Summary sentence: ```Theodule solicits investors for Creative Capital by guaranteeing a 100% re

In [6]:

true_input = tokenize(true_input_text, return_tensors='pt')
false_input = tokenize(false_input_text, return_tensors='pt')

True_idx = tokenize.encode("Yes", add_special_tokens=False)[0]
False_idx = tokenize.encode("No", add_special_tokens=False)[0]

In [7]:
true_output = model(**true_input)
true_logits = true_output.logits

In [8]:
prob = torch.softmax(true_logits[:, -1, :], dim=1)
yes_prob = prob[:, True_idx].item()
no_prob = prob[:, False_idx].item()
print(yes_prob, no_prob)
print(yes_prob > no_prob)

0.6545732617378235 0.1053808405995369
True


In [9]:
false_output = model(**false_input)
false_logits = false_output.logits
false_prob = torch.softmax(false_logits[:, -1, :], dim=1)
yes_false_prob = false_prob[:, True_idx].item()
no_false_prob = false_prob[:, False_idx].item()
print(yes_false_prob, no_false_prob)
print(yes_false_prob > no_false_prob)


0.6491807699203491 0.12524646520614624
True


In [4]:
true_input_text = f"""
Question: Can cat fly?
Answer:
"""

In [5]:
true_input = tokenize(true_input_text, return_tensors='pt')

In [8]:
True_idx = tokenize.encode("Yes", add_special_tokens=False)[0]
False_idx = tokenize.encode("No", add_special_tokens=False)[0]

In [9]:
true_output = model(**true_input)
true_logits = true_output.logits
prob = torch.softmax(true_logits[:, -1, :], dim=1)
yes_prob = prob[:, True_idx].item()
no_prob = prob[:, False_idx].item()
print(yes_prob, no_prob)
print(yes_prob > no_prob)

0.00031695698271505535 0.011991030536592007
False


In [27]:
true_summary = "According to the Commission's complaint, the defendants raised at least $23.4 million from thousands of investors in the Haitian-American community nationwide through a network of purported investment clubs Theodule directs investors to form."
true_text = """The Defendants have engaged in a fraudulent Ponzi scheme primarily targeting the US Haitian community since at least November 2007.This includes net transfers of at least $1.7 million to his personal bank accounts, cash withdrawals of more than $1.5 million and more than $600,000 for apparent personal expenses such as two luxury vehicles, credit card bills, a wedding payment, and a house down payment.21. The investment clubs pool investor funds and send them to Creative Capital for a 90-day period, during which Theodule purportedly trades stocks and options on behalf of the investment club members.Page 2 of 10 
$15.2 million collected from new investors in typical Ponzi scheme fashion.Page 4 of 10 
made millionaires out of a significant number of people in the time it had taken her to decide to invest, and pressured her to liquidate the equity in her home to invest with him.14. Theodule ingratiates himself with investors by claiming he recently decided to offer his investment expertise to help build wealth in the Haitian community.He also tells investors he uses part of his trading profits to fund start-up businesses in the Haitian community, as well as business projects in Haiti and Sierra Leone.15. The Defendants primarily attract investors through word-of-mouth, and Theodule makes his representations during face-to-face meetings in which he touts his ability to double investor funds in just 90 days.18. Since the commencement of the investment scheme, the Defendants have raised more than $23.4 million from thousands of investors nationwide.2, From at least November 2007 to the present, Theodule, directly and through the Companies, has raised at least $23.4 million from thousands of investors in an ongoing fraud and Ponzi scheme targeting mostly Haitian and Haitian-American investors nationwide."""

false_summary = " According to the Commission's complaint, the defendants raised at least $789 billion from thousands of investors in the French community nationwide through a network of purported investment clubs Theodule directs investors to form."

false_text = true_text

In [28]:
true_input_text = f"""
As a compliance officer at a financial institution, you're tasked with evaluating the accuracy of a summary sentence based on its alignment with source sentences from a financial document. Consider the following criteria carefully:

1. The summary accurately reflects the content of the source sentences, especially numerical information.
2. All named entities in the summary are present in the source sentences.
3. Relationships between entities in the summary are consistent with those in the source sentences.
4. The directional flow of relationships among named entities matches between the summary and source sentences.
5. There are no factual discrepancies between the summary and source sentences.
6. The summary does not introduce any entities not found in the source sentences.

Your job is to determine if the summary adheres to these criteria. Answer "Yes" if it does, or "No" if it doesn't.

Summary sentence: ```{true_summary}```

Source sentences: ```{true_text}```

Final Answer (Yes/No only): 
"""
false_input_text = f"""
As a compliance officer at a financial institution, you're tasked with evaluating the accuracy of a summary sentence based on its alignment with source sentences from a financial document. Consider the following criteria carefully:

1. The summary accurately reflects the content of the source sentences, especially numerical information.
2. All named entities in the summary are present in the source sentences.
3. Relationships between entities in the summary are consistent with those in the source sentences.
4. The directional flow of relationships among named entities matches between the summary and source sentences.
5. There are no factual discrepancies between the summary and source sentences.
6. The summary does not introduce any entities not found in the source sentences.

Your job is to determine if the summary adheres to these criteria. Answer "Yes" if it does, or "No" if it doesn't.

Summary sentence: ```{false_summary}```

Source sentences: ```{true_text}```

Final Answer (Yes/No only): 
"""

In [1]:
import openai

In [29]:
print(true_input_text)


As a compliance officer at a financial institution, you're tasked with evaluating the accuracy of a summary sentence based on its alignment with source sentences from a financial document. Consider the following criteria carefully:

1. The summary accurately reflects the content of the source sentences, especially numerical information.
2. All named entities in the summary are present in the source sentences.
3. Relationships between entities in the summary are consistent with those in the source sentences.
4. The directional flow of relationships among named entities matches between the summary and source sentences.
5. There are no factual discrepancies between the summary and source sentences.
6. The summary does not introduce any entities not found in the source sentences.

Your job is to determine if the summary adheres to these criteria. Answer "Yes" if it does, or "No" if it doesn't.

Summary sentence: ```According to the Commission's complaint, the defendants raised at least $2

In [38]:
print(false_input_text)


As a compliance officer at a financial institution, you're tasked with evaluating the accuracy of a summary sentence based on its alignment with source sentences from a financial document. Consider the following criteria carefully:

1. The summary accurately reflects the content of the source sentences, especially numerical information.
2. All named entities in the summary are present in the source sentences.
3. Relationships between entities in the summary are consistent with those in the source sentences.
4. The directional flow of relationships among named entities matches between the summary and source sentences.
5. There are no factual discrepancies between the summary and source sentences.
6. The summary does not introduce any entities not found in the source sentences.

Your job is to determine if the summary adheres to these criteria. Answer "Yes" if it does, or "No" if it doesn't.

Summary sentence: ```Theodule solicits investors for Creative Capital by guaranteeing a 80% ret

In [39]:
from openai import OpenAI
client = OpenAI(api_key = openai_api_key)

completion = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {"role": "user", "content": true_input_text}
  ],
  max_tokens=1
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Yes', role='assistant', function_call=None, tool_calls=None)


In [40]:
from openai import OpenAI
client = OpenAI(api_key = openai_api_key)

completion = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {"role": "user", "content": false_input_text}
  ],
  max_tokens=1
)

print(completion.choices[0].message)

ChatCompletionMessage(content='No', role='assistant', function_call=None, tool_calls=None)


In [16]:
from openai import OpenAI
client = OpenAI(api_key = openai_api_key)

completions = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "text": true_input_text}],
)

text = completions.choices[0].text.strip()
logprobs = completions.choices[0].logprobs.token_logprobs
tokens = logprobs.keys()
probs = logprobs.values()

for token, prob in zip(tokens, probs):
    print(f"Token: {token}, Log Probability: {prob}")

BadRequestError: Error code: 400 - {'error': {'message': "Additional properties are not allowed ('text' was unexpected) - 'messages.0'", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [27]:
logprobs

[-3.410423]

In [28]:
text

'`'

In [29]:
completions.choices

[<OpenAIObject at 0x2685fe4fb80> JSON: {
   "text": " `",
   "index": 0,
   "logprobs": {
     "tokens": [
       " `"
     ],
     "token_logprobs": [
       -3.410423
     ],
     "top_logprobs": [
       {
         " `": -3.410423,
         "1": -1.379173,
         "0": -1.8244855,
         " Yes": -2.0432355,
         " No": -3.441673
       }
     ],
     "text_offset": [
       2965
     ]
   },
   "finish_reason": "length"
 }]

In [30]:
completions = openai.Completion.create(
    engine="davinci-002",
    prompt=false_input_text,
    max_tokens=1,
    n=1,
    logprobs=10,
)

text = completions.choices[0].text.strip()
logprobs = completions.choices[0].logprobs.token_logprobs
tokens = logprobs.keys()
probs = logprobs.values()

for token, prob in zip(tokens, probs):
    print(f"Token: {token}, Log Probability: {prob}")

AttributeError: 'list' object has no attribute 'keys'

In [31]:
completions.choices

[<OpenAIObject at 0x26860317220> JSON: {
   "text": "11",
   "index": 0,
   "logprobs": {
     "tokens": [
       "11"
     ],
     "token_logprobs": [
       -6.108762
     ],
     "top_logprobs": [
       {
         "1": -1.3665743,
         "0": -1.7415743,
         " Yes": -2.1165743,
         " `": -3.3118868,
         " No": -3.4525118
       }
     ],
     "text_offset": [
       2952
     ]
   },
   "finish_reason": "length"
 }]